In [ ]:
import time

import numpy as np
import torch

from carle.env import CARLE
from carle.mcl import CornerBonus, AE2D, RND2D, PufferDetector, SpeedDetector

import matplotlib.pyplot as plt

In [ ]:
# performance when running on cpu and under Life rules

def performance_run(**kwargs):
    """
    
    """    
    
    max_steps = kwargs["max_steps"] if "max_steps" in kwargs.keys() else 32
    reward_wrappers = kwargs["wrappers"] if "wrappers" in kwargs.keys() else []
    rule_string = kwargs["rule_string"] if "rule_string" in kwargs.keys() else "B3/S23"
    verbose = kwargs["verbose"] if "verbose" in kwargs.keys() else False
    my_device = kwargs["device"] if "device" in kwargs.keys() else "cpu"
     
    dimensions = [64, 128, 192, 256, 384, 512]
    vectorization = [1, 2, 4, 8, 16, 32]

    steps_per_s = np.zeros((len(dimensions), len(vectorization)))
    updates_per_s = np.zeros((len(dimensions), len(vectorization)))

    for ii, dim in enumerate(dimensions):

        for jj, instances in enumerate(vectorization):

            env = CARLE(height=dim, width=dim, instances=instances, device=my_device)
            env.rules_from_string(rule_string)
            
            for wrapper in reward_wrappers:
                env = wrapper(env)
            
            obs = env.reset()
            action = torch.zeros(instances, 1, 64, 64).to(torch.device(my_device))

            # action is a simple plus sign
            action[:,:,32,:] = 1.0
            action[:,:,:,32] = 1.0

            t0 = time.time()

            for kk in range(max_steps):

                o, r, d, i = env.step(action)

            t1 = time.time()

            steps_per_second = max_steps / (t1-t0) 
            steps_per_s[ii,jj] = steps_per_second
            updates_per_s[ii, jj] = steps_per_second * instances

            if verbose:
                
                print(f"{max_steps} steps completed in {t1-t0} seconds on {my_device}")
                print(f"{instances} instances, dimension: {dim}, steps per second: {steps_per_second}")
                
    return steps_per_s, updates_per_s

def performance_heatmap(blank_per_s, **kwargs):
    
    x_tick_labels = kwargs["x_tick_labels"] if "x_tick_labels" in kwargs.keys() else None
    y_tick_labels = kwargs["y_tick_labels"] if "y_tick_labels" in kwargs.keys() else None
    
    x_label = kwargs["x_label"] if "x_label" in kwargs.keys() else ""
    y_label = kwargs["y_label"] if "y_label" in kwargs.keys() else ""
    
    title = kwargs["title"] if "title" in kwargs.keys() else "Default Title ;)"
    
    my_font_size=32
    
    fig = plt.figure(figsize=(12,12))
    
    plt.imshow(blank_per_s, cmap="inferno")
    my_cmap = plt.get_cmap("inferno")
    
    if x_tick_labels is not None:
        x_tick_locations = [elem for elem in range(len(x_tick_labels))]
        plt.xticks(x_tick_locations, x_tick_labels, fontsize=my_font_size-10)
    
    if y_tick_labels is not None:
        y_tick_locations = [elem  for elem in range(len(y_tick_labels))]
        plt.yticks(y_tick_locations, y_tick_labels, fontsize=my_font_size-10)
    
    plt.xlabel(x_label, fontsize=my_font_size)
    plt.ylabel(y_label, fontsize=my_font_size)
    plt.title(title, fontsize=my_font_size+6)
    
    # add numbers
    for ii in range(blank_per_s.shape[0]):
        for jj in range(blank_per_s.shape[1]):
            
            x = jj - 0.3
            y = ii + 0.05
            my_text = f"{blank_per_s[ii,jj]:.0f}"
            
            my_color = my_cmap(round(1.0 - (blank_per_s[ii,jj]/np.max(blank_per_s))))
            
            #print(my_color, blank_per_s[ii,jj] / np.max(blank_per_s))
            plt.text(x+.02, y, my_text, fontsize=my_font_size-8, c=my_color)
            
    return fig

In [ ]:
for my_device in ["cpu", "cuda:0", "cuda:1"]:

    steps_per_s, updates_per_s = performance_run(verbose=True, max_steps=2048, device=my_device)

    fig1 = performance_heatmap(steps_per_s, \
                              x_tick_labels=[1, 2, 4, 8, 16, 32],\
                              y_tick_labels=[64, 128, 192, 256, 384, 512],\
                              x_label="Vectorization",\
                              y_label="CA dimension",\
                              title=f"CARLE steps per second on {my_device}")


    fig2 = performance_heatmap(updates_per_s, \
                              x_tick_labels=[1, 2, 4, 8, 16, 32],\
                              y_tick_labels=[64, 128, 192, 256, 384, 512],\
                              x_label="Vectorization",\
                              y_label="CA dimension",\
                              title=f"CARLE updates per second on {my_device}")

    plt.show()

In [ ]:
fig1 = performance_heatmap(steps_per_s, \
                          x_tick_labels=[1, 2, 4, 8, 16, 32],\
                          y_tick_labels=[64, 128, 192, 256, 384, 512],\
                          x_label="Vectorization",\
                          y_label="CA dimension",\
                          title="CARLE steps per second on CPU")
          
    
fig2 = performance_heatmap(updates_per_s, \
                          x_tick_labels=[1, 2, 4, 8, 16],\
                          y_tick_labels=[64, 128, 192, 256, 384, 512],\
                          x_label="Vectorization",\
                          y_label="CA dimension",\
                          title="CARLE updates per second on CPU")
                         
plt.show()